In [1]:
# Notebook for looking at global deaths
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)

import requests

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
# locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))


import os
import math

from datetime import date
import datetime

saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]
def plotWithMean(ax,x,y,meanWidth):
    firstLine = ax.plot(x,y,'.:',markersize=2,linewidth=0.5)
    firstColor = firstLine[0].get_color()
    ax.plot(rnTime(x,meanWidth),rnMean(y,meanWidth),color=firstColor)

saveFigures is set to: True
Done loading packages


In [2]:
dataPath = './data/OWID_excess.csv'

df = pd.read_csv(dataPath)

df.date = pd.to_datetime(df.date)
dkdf = df[df.location == 'Denmark']

# Also load COVID data
dataPath = './data/OWID_data.csv'

dfData = pd.read_csv(dataPath)

dfData.date = pd.to_datetime(dfData.date)
dkdfData = dfData[dfData.location == 'Denmark']



In [3]:

# testdf = pd.read_csv("C:/Users/rakrpe/Documents/PandemiX/GithubRepos/excess-mortality/elife2021/frozen-data/world_mortality.csv")
# testdf[testdf.country_name == 'Denmark']
# df
# dkdf

In [4]:

# # curdf
# count2021 
# # curdf.tail()
# 
# # dates2021 = curDates[curDates >= np.datetime64('2021-01-01')]
# # numData2021 =(~curdf.deaths_2021_all_ages.isna()).sum()
# # dates2021 = dates2021.iloc[:numData2021]
# allDates,allCount
# curWidths = np.diff(allDates).astype('timedelta64[D]')
# len(curWidths)
# widthToUse
# # widthToUse = []
# # for k in range(len(curWidths)):
# #     widthToUse.append(curWidths[k]/2+curWidths[k+1]/2)
# # pd.to_datetime(allDates)


#     # curWidths = np.diff(allDates).astype('timedelta64[D]')
#     # ax1.bar(allDates,allCount,color='grey',edgecolor='k',width=[30,curWidths])
# fig,ax1 = plt.subplots()
# stepDates = np.append(np.append(allDates[0]-np.timedelta64(curWidth,'D'),allDates),allDates[-1]+np.timedelta64(curWidth,'D'))
# stepVals = np.append(np.append(avgComp[0],avgComp),avgComp[-1]) 
# ax1.step(stepDates,stepVals,color='k',where='mid')

In [5]:


# Get all possible locations
posLoc = df.location.unique()

# Loop through them
# for curLoc in posLoc[:4]:
for curLoc in posLoc:
    
    # Get current location-dataframe
    curdf = df[df.location == curLoc]
    curdfData = dfData[dfData.location == curLoc]
    
    # Make figure
    fig,ax1 = plt.subplots()

    # Split the dates
    curDates = curdf.date
    dates2021 = curDates[curDates >= np.datetime64('2021-01-01')]
    dates2020 = curDates[curDates < np.datetime64('2021-01-01')]
    
    # Only take days in 2021 for which there is data
    numData2021 =(~curdf.deaths_2021_all_ages.isna()).sum()
    dates2021 = dates2021.iloc[:numData2021]
    numData2020 =(~curdf.deaths_2020_all_ages.isna()).sum()
    dates2020 = curDates.iloc[:numData2020]

    # Get data, without nan
    count2021 = curdf.deaths_2021_all_ages.dropna()
    count2020 = curdf.deaths_2020_all_ages.dropna()
    
    # Collect dates and counts in arrays
    # allDates = np.append(dates2020,dates2021)
    allCount = np.append(count2020,count2021)
    allDates = curdf.date.iloc[:len(allCount)].values
    
    # Determine the time scale, and decide on bar width
    curTimeUnit = curdf.time_unit.iloc[0]
    curWidth = 1
    if (curTimeUnit == 'monthly'):
        curWidth = 31
    if (curTimeUnit == 'weekly'):
        curWidth = 7
    
    ax1.bar(allDates,allCount,color='grey',width=np.timedelta64(curWidth,'D'))
    # ax1.bar(allDates,allCount,color='grey',edgecolor='k',width=np.timedelta64(curWidth,'D'))
    
    

    avgDates = np.append(dates2020,dates2020+np.timedelta64(371,'D'))
    avgCount = curdf.average_deaths_2015_2019_all_ages.dropna()
    avgCountFull = np.append(avgCount,avgCount)

    numData = len(allDates)
    avgComp = avgCountFull[:numData]


    allCountPos = allCount * ((allCount - avgComp) > 0  )
    allCountPos[allCountPos == 0] = np.nan

    allCountNeg = allCount * ((allCount - avgComp) < 0  )
    allCountNeg[allCountNeg == 0] = np.nan

    allCountNeg = allCount * ((allCount - avgComp) < 0  )
    curIndex = (allCountNeg == 0)
    allCountNeg[curIndex]  = avgComp[curIndex]
    allCountNeg


    ax1.bar(allDates,allCount,width=np.timedelta64(curWidth,'D'),color='r',label='Excess')
    # ax1.bar(allDates,avgComp,width=np.timedelta64(curWidth,'D'),edgecolor='k',linewidth=3)
    # ax1.step(allDates,avgComp,color='k',where='mid')
    # stepDates = np.append(np.append(allDates[0],allDates),allDates[-1])
    stepDates = np.append(np.append(allDates[0]-np.timedelta64(int(curWidth/2),'D'),allDates),allDates[-1]+np.timedelta64(int(curWidth/2),'D'))
    stepVals = np.append(np.append(avgComp[0],avgComp),avgComp[-1]) 
    ax1.step(stepDates,stepVals,color='k',where='mid',label='Average 2015-2019')
    ax1.bar(allDates,avgComp,width=np.timedelta64(curWidth,'D'),color='g',label='Negative excess')
    ax1.bar(allDates,allCountNeg,width=np.timedelta64(curWidth,'D'),color='w')


    # ax1.plot(avgDates,avgCountFull,color='k')

    # ax1.plot(avgDates,avgCountFull*np.nan,color='k',label='Average 2015-2019')
    # ax1.bar(allDates,(allCount-avgComp),bottom=avgComp,width=np.timedelta64(7,'D'),color='r')
    # ax1.bar(allDates,(avgComp-allCount),bottom=allCount,width=np.timedelta64(7,'D'),color='g')

    ax1.set_ylim([min(allCount)*0.8,max(allCount)*1.1])
    # ax1.set_xlim([allDates[0],allDates[-1]])
    # ax1.set_xlim(right=allDates[-1])
    # ax1.set_xlim(left=np.datetime64('2020-01-01'))
    # ax1.set_xlim(left=allDates[0]-np.timedelta64(int(curWidth/2),'D'))
    # ax1.set_xlim(right=allDates[-1]+np.timedelta64(int(curWidth/2),'D'))
    
    curXticks = np.arange(np.datetime64('2020-01'),np.datetime64('2021-10'),np.timedelta64(3,'M'))
    ax1.set_xticks(curXticks)
    ax1.set_xlim([curXticks[0],np.datetime64('2021-09')])

    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

    ax1.set_ylabel('Mortality count')
    ax1.legend()
    ax1.grid(axis='x',zorder=-1)
    
    # numDeaths2020 = dkdf['deaths_2020_all_ages'].sum()
    # dkdf['average_deaths_2015_2019_all_ages'].sum() 
    
    
    
    excessMort = (allCount-avgComp)
    totExcess = excessMort.sum()
    
    curTitle = curLoc + f', Total excess: {totExcess:.0f}'
    ax1.set_title(curTitle)

    if saveFigures:
        plt.savefig('figs/ExcessMortality/'+curLoc)
        
    plt.close()
    
    try:
        fig,ax2 = plt.subplots()
        
        thisDates = pd.to_datetime(curdfData.date)
        ax2.plot(allDates,excessMort*0,color='k')
        ax2.plot(thisDates,curdfData['new_deaths'],'.:b',markersize=2,linewidth=0.5)
        ax2.plot(rnTime(thisDates,7),rnMean(curdfData['new_deaths'],7),'b')
        # ax2.plot(allDates,excessMort/curWidth,'.-',color='r')
        ax2.bar(allDates,excessMort/curWidth,color='r',width=np.timedelta64(curWidth,'D'))
        # ax2.plot(thisDates,curdfData['excess_mortality'],'*')

        ax2.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

        if saveFigures:
            plt.savefig('figs/ExcessMortalityCompareCovid/'+curLoc)
            
        plt.close()
    except:
        print('Could not make second figure for '+curLoc)
        continue

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:

try:
    ax2.plot(rnTime(thisDates,7),rnMean(curdfData['new_deaths'],7),'b')
except:
    2+2

In [ ]:
# curdfData['date'] = pd.to_datetime(curdfData['date'])

thisDates = pd.to_datetime(dkdfData.date)

fig,ax1 = plt.subplots()

ax1.plot(allDates,excessMort*0,color='k')
ax1.plot(thisDates,dkdfData['new_deaths'],'.:b',markersize=2,linewidth=0.5)
ax1.plot(rnTime(thisDates,7),rnMean(dkdfData['new_deaths'],7),'b')
# ax1.plot(allDates,excessMort/7,'.-',color='r')
ax1.bar(allDates,excessMort/7,color='r',width=np.timedelta64(7,'D'))
# ax1.plot(thisDates,curdfData['excess_mortality'],'*')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

# if saveFigures:
#     plt.savefig('figs/ExcessMortalityCompareWithCOVID_DK')

NameError: name 'pd' is not defined